https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=VJ76KiP_dLn-

https://colab.research.google.com/drive/1uSlWtJdZmLrI3FCNIlUHFxwAJiSu2J0-

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools
import math
from torch.jit import script, trace
from torch import optim

print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

1.7.1+cu110
cuda


In [2]:
import tensorflow as tf
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer # kobert의 get_tokenizer와 같이 쓰기 위함
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# 데이터1

In [3]:
corpus_name = 'chatData'
corpus = os.path.join(r'C:\Users\admin\jupyter\pytorch\chatbot', corpus_name)

In [4]:
model, vocab  = get_pytorch_kobert_model()

using cached model
using cached model


In [5]:
datafile = os.path.join(corpus, 'kotest.txt')

PAD_token = 0
SOS_token = 1
EOS_token = 2
CLS_token = 101
SEP_token = 102
tokenizer = get_tokenizer() 
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
transform = nlp.data.BERTSentenceTransform(tokenizer=tok, max_seq_length=64, pad=False, pair=False)

class Voc:
    def __init__(self, name):
        self.name = name
        self.tokens = []
        
    def addSentence(self, sentence): # sentence : 잘 마실게
        tokens = tok(sentence)
        self.tokens = tokens

        return tokens            

using cached model


In [6]:
MAX_LENGTH = 20 

def readVocs(datafile, corpus_name):
    print('Reading lines...')
    lines = open(datafile, encoding='utf-8').read().strip().split('\n')
    pairs = [l.strip().split('[SEP]') for l in lines]
    #print(pairs[0])
    voc = Voc(corpus_name)
    return voc, pairs # voc : 문서 단어집합 / pairs : 문장 쌍 집합

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name) # voc : 단어집합, pairs : 질문 쌍
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    for pair in pairs:
        sentence1 = voc.addSentence(pair[0].strip())
        #print(sentence1)
        sentence2 = voc.addSentence(pair[1].strip())
    return voc, pairs

save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)

for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 127856 sentence pairs
Trimmed to 114332 sentence pairs
['잘 마실게.', '오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서..']
['오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서..', '무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래.']
['무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래.', '하..']
['언니 과외 마쳤어요? 오늘은 늦게까지 했네요.', '기말고사라..내가 하나 들어줘요?']
['기말고사라..내가 하나 들어줘요?', '괜찮아요.']
['성적가지고 해리한테 너무 그러지 마. 애가 엄청 스트레스 받는 모양이던데. 아직 초등학생인데 점수 좀 못 받으면 뭐 어떻다고.', '스트레스 받는 애 성적이 저 모양이야?']
['스트레스 받는 애 성적이 저 모양이야?', '지도 잘하고 싶겠지. 지라고 못하고 싶겠어?']
['또 비교한다. 당신 그렇게 비교하는 게 애한테 얼마나 안좋은.', '아 몰라! 비교가 안되야 비굘 안하지. 하여튼 맘에 안 들어. 나같음 자존심이 상해서라도 열심히 하겠는데..아우. 진짜 보고 있음 열불터져서..']
['아 몰라! 비교가 안되야 비굘 안하지. 하여튼 맘에 안 들어. 나같음 자존심이 상해서라도 열심히 하겠는데..아우. 진짜 보고 있음 열불터져서..', '내가 보기엔']
['내가 보기엔', '나가서 냉수 한잔만 떠다줘. 아우. 괜히 또 열받네.']


In [7]:
def indexesFromSentence(voc, sentence):
    token_to_encode = transform([sentence]) # token_to_encode 튜플형식으로 반환
    return token_to_encode[0].tolist() # 0번째를 추출하여 리스트형식으로 변환해서 반환

def zeroPadding(l, fillvalue=PAD_token):
    # print(list(itertools.zip_longest(*l, fillvalue=fillvalue))) : (1957, 2, 2, 2, 0)
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def inputVar(l, voc): # l : 응. 새벽에도 일찍 나갔어. 온다간다 말도없이.
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l] 
    #print(indexes_batch) # : [[5027, 1239, 9433, 2], [5951, 4686, 1476, 2], [1116, 5309, 2], [319, 2], [186, 2]]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch]) 
    # print(lengths) : tensor([4, 4, 3, 2, 2])
    padList = zeroPadding(indexes_batch)
    # print(padList) : [[5027, 1239, 9433, 2], [5951, 4686, 1476, 2], [1116, 5309, 2, 0], [319, 2, 0, 0], [186, 2, 0, 0]]
    padVar = torch.LongTensor(padList)
    return padVar, lengths

def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch]) 
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList) 
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(transform([x[0]])[0].tolist()), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        #print(pair[0])
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[   2,    2,    2,    2,    2],
        [3093, 1698, 2110, 3958, 2145],
        [6323, 1804, 3093, 3097, 7797],
        [5778, 6386, 6113, 6844,  633],
        [ 517, 5377, 6277,  517,    3],
        [5961, 2145, 7659,   54,    0],
        [5384, 7141, 6999,    3,    0],
        [2081, 6857,  633,    0,    0],
        [6006,  633,    3,    0,    0],
        [1185, 3647,    0,    0,    0],
        [3759, 7354,    0,    0,    0],
        [7086, 1612,    0,    0,    0],
        [4128, 6079,    0,    0,    0],
        [5330, 1730,    0,    0,    0],
        [3175, 7848,    0,    0,    0],
        [6553, 1598,    0,    0,    0],
        [4986, 5760,    0,    0,    0],
        [ 517, 5384,    0,    0,    0],
        [  54, 5850,    0,    0,    0],
        [4244,  517,    0,    0,    0],
        [7096,   54,    0,    0,    0],
        [5886,    3,    0,    0,    0],
        [5928,    0,    0,    0,    0],
        [2266,    0,    0,    0,    0],
        [7096,    0,    

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, 
                          dropout = (0 if n_layers == 1 else dropout), bidirectional=True)
        
    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq) # input_seq : shape=(max_length, batch_size), input_variable
        # print(input_lengths) : =lengths
        # print(embedded.shape) : torch.Size([10, 64, 500]) [max_length, batch_size, hidden_size(은닉상태 크기)]
        
        # nn.utils.rnn.pack_padded_sequence : 패딩연산처리 쉽게하기 위해 중간에 빈공간 제거(형태 : tensor)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths.cpu()) # input_lengths : shape=(batch_size)
        #print(packed.batch_sizes)
        # print(packed.batch_sizes) : tensor([64, 64, 64, 58, 52, 45, 38, 17,  8,  2])
        
        outputs, hidden = self.gru(packed, hidden) # 입력hidden : shape=(n_layers * num_directions, batch_size, hidden_size)
        # print(outputs.batch_sizes) : tensor([64, 64, 63, 52, 47, 34, 24, 18, 12,  6])
        # print(hidden.shape) : torch.Size([4, 64, 500]) [층 * 양방향이면2 아니면1, batch_size, hidden_size]
        
        # nn.utils.rnn.pad_packed_sequence : 패딩연산이 끝난 것을 다시 원래대로 (형태 : torch)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        #print(outputs.shape)
        # print(outputs.shape)# : torch.Size([10, 64, 1000]) # [max_length, batch_size, hidden_size(양방향으로 진행했으면 *2)]
        
        # 양방향 GRU의 출력을 합산합니다
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # print(outputs.shape) : torch.Size([10, 64, 500])
        
        # hidden : GRU의 최종 은닉 상태
        return outputs, hidden

In [9]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, 'is not an appropriate attention method.')
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))
            
    # 가중치 계산을 dot-product로 계산
    def dot_score(self, hidden, encoder_output):
        # print(torch.sum(hidden * encoder_output, dim=2).shape) : torch.Size([10, 64]) 10개 생성[max_length, batch_size]
        return torch.sum(hidden * encoder_output, dim=2)
    
    # 
    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        # print(energy.shape) : torch.Size([10, 64, 500]) 10개 생성[max_length, batch_size, hidden_size]
        
        # print(torch.sum(hidden * energy, dim=2).shape) : torch.Size([10, 64]) 10개 생성[max_length, batch_size]
        return torch.sum(hidden * energy, dim=2)
    
    
    def concat_score(self, hidden, encoder_output):
        # cat : 합칠 때 차원은 2차원으로 / expand : 확장
        # Tanh 함수는 함수값을 [-1, 1]로 제한시킴
        # print((hidden.expand(encoder_output.size(0), -1, -1).shape)) : torch.Size([10, 64, 500])
        # print(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2).shape) : torch.Size([10, 64, 1000])
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        # print(energy.shape) : torch.Size([10, 64, 500]) 10개 생성[max_length, batch_size, hidden_size]
        return torch.sum(self.v * energy, dim=2)
    
    def forward(self, hidden, encoder_outputs):
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
            
        attn_energies = attn_energies.t() # t() : 행과 열을 바꿔서 저장[1, 2, 3], [4, 5, 6] -> [1, 4, 7], [2, 5, 8]
        
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [10]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # 참조를 보존해 둡니다
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # 레이어를 정의합니다
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # 주의: 한 단위 시간에 대해 한 단계(단어)만을 수행합니다
        # 현재의 입력 단어에 대한 임베딩을 구합니다   
        #print(input_step)
        embedded = self.embedding(input_step) # input_step : 입력 시퀀스 배치에 대한 한 단위 시간(한 단어). shape=(1, batch_size)
        embedded = self.embedding_dropout(embedded)
        #print(embedded.shape)
        # print(embedded.shape) : torch.Size([1, 64, 500])
        
        # 양방향x
        # last_hidden : GRU의 마지막 은닉 레이어. shape=(n_layers * num_directions, batch_size, hidden_size)
        # print(last_hidden.shape) : torch.Size([2, 64, 500]) 
        rnn_output, hidden = self.gru(embedded, last_hidden) 
        # print(rnn_output.shape) : torch.Size([1, 64, 500])
        # print(hidden.shape) : torch.Size([2, 64, 500])

        # attention 가중치
        attn_weights = self.attn(rnn_output, encoder_outputs) # encoder_outputs : 인코더 모델 출력 shape=(max_length, batch_size, hidden_size)
        # print(attn_weights.shape) : torch.Size([64, 1, 10]) 

        # 인코더 출력에 어텐션을 곱하여 새로운 context vector생성
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # print(context.shape) : torch.Size([64, 1, 500])

        rnn_output = rnn_output.squeeze(0) # print(rnn_output.shape) : torch.Size([64, 500])
        context = context.squeeze(1) # print(context.shape) : torch.Size([64, 500])
        concat_input = torch.cat((rnn_output, context), 1) # print(concat_input.shape) : torch.Size([64, 1000])
        concat_output = torch.tanh(self.concat(concat_input))
        # print(concat_output.shape) : torch.Size([64, 500])

        # output : 각 단어가 디코딩된 시퀀스에서 다음 단어로 사용되었을 때 적합할 확률을 나타내는 정규화된 softmax 텐서. 
        # shape=(batch_size, voc.num_words)
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)

        return output, hidden

In [11]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [12]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer, decoder_optimizer,
         batch_size, clip, max_length = MAX_LENGTH):
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    
    loss = 0
    print_losses = []
    n_totals = 0
    
    # EncoderRNN의 forward부분 실행
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
    
    # 초기 디코더 입력을 생성(각 문장을 SOS 토큰으로 시작)
    decoder_input = torch.LongTensor([[CLS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    
    # 디코더의 초기 은닉 상태를 인코더의 마지막 은닉 상태로
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    
    # teacher_forcing : Decoder부분에서 앞 단어가 잘못 추측되었을 경우 뒤에도 달라지니 정답을 입력해 주는 것
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        for t in range(max_target_len):
            # LuongAttnDecoderRNN의 forward로 실행
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
             # Teacher forcing 사용: 다음 입력을 현재의 목표로 둡니다
            decoder_input = target_variable[t].view(1, -1)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            # Teacher forcing 미사용: 다음 입력을 디코더의 출력으로 둡니다
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
            
    loss.backward()
    
    # clip_grad_norm_: 그라디언트를 제자리에서 수정합니다
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return sum(print_losses) / n_totals

In [13]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # 각 단계에 대한 배치 설정
    # batch2TrainData : return inp, lengths, output, mask, max_target_len
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]


    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1


    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        torch.cuda.empty_cache() # GPU 캐시 데이터 삭제
        training_batch = training_batches[iteration - 1]
        
        input_variable, lengths, target_variable, mask, max_target_len = training_batch


        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        
        print_loss += loss


        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Checkpoint를 저장
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, 'sum', '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [14]:
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 768
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

loadFilename = None
checkpoint_iter = 40000
loadFilename = os.path.join(save_dir, model_name, corpus_name, 'sum',
                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                            '{}_checkpoint.tar'.format(checkpoint_iter))


# loadFilename이 제공되는 경우에는 모델을 불러옵니다
if loadFilename:
    # 모델을 학습할 때와 같은 기기에서 불러오는 경우
    checkpoint = torch.load(loadFilename)
    # GPU에서 학습한 모델을 CPU로 불러오는 경우
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')

embedding = nn.Embedding(len(vocab), hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, len(vocab), decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)

encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [15]:
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 80000
print_every = 1
save_every = 10000

# Dropout 레이어를 학습 모드로 둡니다
encoder.train()
decoder.train()

# Optimizer를 초기화합니다
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)


for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# 학습 단계를 수행합니다
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...


<ipython-input-11-f319a975feb5>:4: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen/native/IndexingUtils.h:25.)
  loss = crossEntropy.masked_select(mask).mean()


Iteration: 40001; Percent complete: 66.7%; Average loss: 1.8953
Iteration: 40002; Percent complete: 66.7%; Average loss: 2.0724
Iteration: 40003; Percent complete: 66.7%; Average loss: 1.9446
Iteration: 40004; Percent complete: 66.7%; Average loss: 2.0288
Iteration: 40005; Percent complete: 66.7%; Average loss: 2.0800
Iteration: 40006; Percent complete: 66.7%; Average loss: 2.2089
Iteration: 40007; Percent complete: 66.7%; Average loss: nan
Iteration: 40008; Percent complete: 66.7%; Average loss: 1.9975
Iteration: 40009; Percent complete: 66.7%; Average loss: 1.9156
Iteration: 40010; Percent complete: 66.7%; Average loss: 1.8834
Iteration: 40011; Percent complete: 66.7%; Average loss: 1.8664
Iteration: 40012; Percent complete: 66.7%; Average loss: 2.1168
Iteration: 40013; Percent complete: 66.7%; Average loss: 1.9986
Iteration: 40014; Percent complete: 66.7%; Average loss: 1.8030
Iteration: 40015; Percent complete: 66.7%; Average loss: 2.3324
Iteration: 40016; Percent complete: 66.7%; 

Iteration: 40130; Percent complete: 66.9%; Average loss: 1.9492
Iteration: 40131; Percent complete: 66.9%; Average loss: 1.8591
Iteration: 40132; Percent complete: 66.9%; Average loss: 2.0362
Iteration: 40133; Percent complete: 66.9%; Average loss: 1.9347
Iteration: 40134; Percent complete: 66.9%; Average loss: 2.0851
Iteration: 40135; Percent complete: 66.9%; Average loss: 2.1798
Iteration: 40136; Percent complete: 66.9%; Average loss: 1.8994
Iteration: 40137; Percent complete: 66.9%; Average loss: 1.9475
Iteration: 40138; Percent complete: 66.9%; Average loss: 1.8376
Iteration: 40139; Percent complete: 66.9%; Average loss: 1.8528
Iteration: 40140; Percent complete: 66.9%; Average loss: 2.0094
Iteration: 40141; Percent complete: 66.9%; Average loss: 2.3067
Iteration: 40142; Percent complete: 66.9%; Average loss: 1.9017
Iteration: 40143; Percent complete: 66.9%; Average loss: 1.8081
Iteration: 40144; Percent complete: 66.9%; Average loss: nan
Iteration: 40145; Percent complete: 66.9%; 

Iteration: 40259; Percent complete: 67.1%; Average loss: 2.1939
Iteration: 40260; Percent complete: 67.1%; Average loss: 2.0674
Iteration: 40261; Percent complete: 67.1%; Average loss: 2.0579
Iteration: 40262; Percent complete: 67.1%; Average loss: 1.9733
Iteration: 40263; Percent complete: 67.1%; Average loss: 2.1031
Iteration: 40264; Percent complete: 67.1%; Average loss: 2.0523
Iteration: 40265; Percent complete: 67.1%; Average loss: 1.9539
Iteration: 40266; Percent complete: 67.1%; Average loss: 2.0151
Iteration: 40267; Percent complete: 67.1%; Average loss: 2.0096
Iteration: 40268; Percent complete: 67.1%; Average loss: 2.0525
Iteration: 40269; Percent complete: 67.1%; Average loss: 2.0045
Iteration: 40270; Percent complete: 67.1%; Average loss: 2.1589
Iteration: 40271; Percent complete: 67.1%; Average loss: 2.3821
Iteration: 40272; Percent complete: 67.1%; Average loss: 1.9211
Iteration: 40273; Percent complete: 67.1%; Average loss: 1.8513
Iteration: 40274; Percent complete: 67.1

...

Iteration: 59480; Percent complete: 99.1%; Average loss: 1.7681
Iteration: 59481; Percent complete: 99.1%; Average loss: 1.6180
Iteration: 59482; Percent complete: 99.1%; Average loss: nan
Iteration: 59483; Percent complete: 99.1%; Average loss: 1.7588
Iteration: 59484; Percent complete: 99.1%; Average loss: 1.8435
Iteration: 59485; Percent complete: 99.1%; Average loss: 1.6671
Iteration: 59486; Percent complete: 99.1%; Average loss: 1.7873
Iteration: 59487; Percent complete: 99.1%; Average loss: 1.9267
Iteration: 59488; Percent complete: 99.1%; Average loss: 1.8641
Iteration: 59489; Percent complete: 99.1%; Average loss: 1.9657
Iteration: 59490; Percent complete: 99.2%; Average loss: 1.8467
Iteration: 59491; Percent complete: 99.2%; Average loss: nan
Iteration: 59492; Percent complete: 99.2%; Average loss: 1.8099
Iteration: 59493; Percent complete: 99.2%; Average loss: 1.7309
Iteration: 59494; Percent complete: 99.2%; Average loss: 1.6900
Iteration: 59495; Percent complete: 99.2%; Ave

Iteration: 59609; Percent complete: 99.3%; Average loss: 1.6672
Iteration: 59610; Percent complete: 99.4%; Average loss: 1.8449
Iteration: 59611; Percent complete: 99.4%; Average loss: 1.8946
Iteration: 59612; Percent complete: 99.4%; Average loss: 1.8780
Iteration: 59613; Percent complete: 99.4%; Average loss: 1.6763
Iteration: 59614; Percent complete: 99.4%; Average loss: 1.7910
Iteration: 59615; Percent complete: 99.4%; Average loss: 1.6140
Iteration: 59616; Percent complete: 99.4%; Average loss: 1.7976
Iteration: 59617; Percent complete: 99.4%; Average loss: 1.7013
Iteration: 59618; Percent complete: 99.4%; Average loss: 1.8862
Iteration: 59619; Percent complete: 99.4%; Average loss: 1.9446
Iteration: 59620; Percent complete: 99.4%; Average loss: 1.8307
Iteration: 59621; Percent complete: 99.4%; Average loss: 1.6550
Iteration: 59622; Percent complete: 99.4%; Average loss: 1.7160
Iteration: 59623; Percent complete: 99.4%; Average loss: 1.6803
Iteration: 59624; Percent complete: 99.4

Iteration: 59738; Percent complete: 99.6%; Average loss: 1.8603
Iteration: 59739; Percent complete: 99.6%; Average loss: 1.6447
Iteration: 59740; Percent complete: 99.6%; Average loss: 1.9250
Iteration: 59741; Percent complete: 99.6%; Average loss: 1.7279
Iteration: 59742; Percent complete: 99.6%; Average loss: 1.7927
Iteration: 59743; Percent complete: 99.6%; Average loss: 1.7079
Iteration: 59744; Percent complete: 99.6%; Average loss: 1.7696
Iteration: 59745; Percent complete: 99.6%; Average loss: 1.6854
Iteration: 59746; Percent complete: 99.6%; Average loss: 1.8406
Iteration: 59747; Percent complete: 99.6%; Average loss: 1.7282
Iteration: 59748; Percent complete: 99.6%; Average loss: 1.7938
Iteration: 59749; Percent complete: 99.6%; Average loss: 1.7689
Iteration: 59750; Percent complete: 99.6%; Average loss: 1.8909
Iteration: 59751; Percent complete: 99.6%; Average loss: 1.6952
Iteration: 59752; Percent complete: 99.6%; Average loss: 1.8573
Iteration: 59753; Percent complete: 99.6

Iteration: 59867; Percent complete: 99.8%; Average loss: 1.9562
Iteration: 59868; Percent complete: 99.8%; Average loss: 1.6583
Iteration: 59869; Percent complete: 99.8%; Average loss: 1.6811
Iteration: 59870; Percent complete: 99.8%; Average loss: 1.8203
Iteration: 59871; Percent complete: 99.8%; Average loss: 1.7059
Iteration: 59872; Percent complete: 99.8%; Average loss: 1.7134
Iteration: 59873; Percent complete: 99.8%; Average loss: 1.8903
Iteration: 59874; Percent complete: 99.8%; Average loss: 2.0471
Iteration: 59875; Percent complete: 99.8%; Average loss: 1.5926
Iteration: 59876; Percent complete: 99.8%; Average loss: 1.6649
Iteration: 59877; Percent complete: 99.8%; Average loss: 1.8311
Iteration: 59878; Percent complete: 99.8%; Average loss: 1.8517
Iteration: 59879; Percent complete: 99.8%; Average loss: 1.6597
Iteration: 59880; Percent complete: 99.8%; Average loss: 1.6936
Iteration: 59881; Percent complete: 99.8%; Average loss: 1.7891
Iteration: 59882; Percent complete: 99.8

Iteration: 59995; Percent complete: 100.0%; Average loss: 1.7465
Iteration: 59996; Percent complete: 100.0%; Average loss: 1.7223
Iteration: 59997; Percent complete: 100.0%; Average loss: 1.7875
Iteration: 59998; Percent complete: 100.0%; Average loss: 1.5464
Iteration: 59999; Percent complete: 100.0%; Average loss: 1.7493
Iteration: 60000; Percent complete: 100.0%; Average loss: 1.8410


In [16]:
# 탐욕적 디코딩(Greedy decoding) : 각 단계에 대해 단순히 decoder_output 에서 가장 높은 softmax값을 갖는 단어를 선택하는 방식
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):

        # EncoderRNN의 forward부분 실행
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        #print('outputs : ', encoder_outputs.shape)
        #print('hidden : ', encoder_hidden.shape)
               
        # encoder의 마지막 hidden이 decoder의 처음 hidden
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        #print('decoder_hidden : ', decoder_hidden.shape)
        
        # decoder의 처음입력을 SOS로 초기화
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * CLS_token
        #print('decoder_input : ', decoder_input)
        
        # 디코더가 단어를 덧붙여 나갈 텐서를 초기화
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)

        for _ in range(max_length):
            # LuongAttnDecoderRNN의 forward로 실행
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            #print('decoder_output : ', decoder_output.shape)
            #print('decoder_hidden : ', decoder_hidden.shape)
            
            # 가장 가능성 높은 단어 토큰과 그 softmax 점수를 구합니다
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            #print('decoder_scores : ', decoder_scores)
            #print('decoder_input : ', decoder_input)

            # 토큰, 점수 기록
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)

            # 현재의 토큰을 디코더의 다음 입력으로 준비시킵니다(차원을 증가시켜서)
            decoder_input = torch.unsqueeze(decoder_input, 0)

        return all_tokens, all_scores

In [17]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    # indexes_batch : 문장을 단어집합에 저장된 수로 바꾼후 마지막에 EOS추가하는 함수
    token_to_encode = transform([sentence])
    indexes_batch = [token_to_encode[0].tolist()]
    #print(indexes_batch)
    #print('input : ', indexes_batch)
 
    # lengths 텐서를 만듭니다
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    #print(lengths)
    
    # 배치의 차원을 뒤집어서 모델이 사용하는 형태로 만듭니다
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    #print('input : ', input_batch)
    
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    
    # searcher를 이용하여 문장을 디코딩합니다
    G_tokens, scores = searcher(input_batch, lengths, max_length)
    #print(G_tokens)
    #print(type(G_tokens))
    #print(scores)

    # 인덱스 -> 단어    
    #print([tokenizer.convert_ids_to_tokens(token) for token in G_tokens])
#     for token in G_tokens:
#         print(tokenizer.convert_ids_to_tokens([token]))
    
    decoded_words = []
    
    for i in G_tokens:
        decoded_words.append(vocab.idx_to_token[i])
        #print(vocab.idx_to_token[i])
        #print(decoded_words)
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        result = []
        cnt = 0
        try:
            # 입력 문장을 받아옵니다
            input_sentence = input('> ')
            # 종료 조건인지 검사합니다
            if input_sentence == 'exit' or input_sentence == '종료': break
            # 문장을 평가합니다
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 응답 문장을 형식에 맞춰 출력합니다
            output_words[:] = [x for x in output_words if not (x == '[CLS]' or x == '[SEP]' or x == 'PAD' or x == 'SOS' or x == 'EOS')]
            
            for i, text in enumerate(output_words):
                if i == 0:
                    result.append(text[1:])
                else:
                    if text[0] == '▁':
                        cnt += 1
                        temp = result[i - cnt] + output_words[output_words.index(text)].replace(text, text[1:])
                        result.append(temp)
                        result.pop(i - cnt)
                    else:
                        result.append(text)
            
            print('Bot:', ' '.join(result))

        except KeyError:
            print("Error: Encountered unknown word.")

In [18]:
# Dropout 레이어를 평가 모드로 설정합니다
encoder.eval()
decoder.eval()

# 탐색 모듈을 초기화합니다
searcher = GreedySearchDecoder(encoder, decoder)

# 채팅을 시작합니다 (다음 줄의 주석을 제거하면 시작해볼 수 있습니다)
evaluateInput(encoder, decoder, searcher, voc)

> 안녕?
Bot: " ... ... .난 ,신 애 랑 ."?
> 어디야?
Bot: 집 에 . . . 가 서 . . . 가
> 누구야?
Bot: 어 . .왜??? 지 . 지 .
> 이름이 뭐야?
Bot: 뭐 . . . 지 . . . 지 .
> 어디에 살아?
Bot: 회사임원 이없어 서누군가 가 세요 . 도돌아 갈 께 요 .
> 뭐래니
Bot:  끊 지 . 가 떼 먹 어 . , 찜 질
> 종료
